非常に参考になる解説：
https://mosko.tokyo/post/pytorch-rnn/

# RNN


In [5]:
%%bash
wget https://github.com/moskomule/pytorch.learning/blob/master/tutorial/rnn.py

wget: /home/toshinao/anaconda3/envs/base2/lib/libcrypto.so.1.0.0: no version information available (required by wget)
wget: /home/toshinao/anaconda3/envs/base2/lib/libssl.so.1.0.0: no version information available (required by wget)
wget: /home/toshinao/anaconda3/envs/base2/lib/libssl.so.1.0.0: no version information available (required by wget)
--2019-02-09 19:50:12--  https://github.com/moskomule/pytorch.learning/blob/master/tutorial/rnn.py
Resolving github.com (github.com)... 192.30.255.113, 192.30.255.112
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2019-02-09 19:50:12 ERROR 404: Not Found.



In [2]:
%%bash
git add theoretical_consideraiton.ipynb

In [11]:
%%bash
git commit -a -m "check point"

[master 8a90035] check point
 1 file changed, 31 insertions(+), 9 deletions(-)


$$
\begin{align}
\text{時点$t$における$l$番目の(hidden) layerにおけるnode value}=h^{(l)}_t=f
\left(
    W^{(l)}   
    \begin{bmatrix}
        h^{(l-1)}_t\\
        h^{(l)}_{t-1}
    \end{bmatrix}
\right)
\end{align}
$$


* RNNはoption on optionのように考えると理解しやすい

対応は以下のような感じか？

* ひとつめのオプションに対してクーポンが入力される
    * クーポンはembedded vectorになっている「ワード」なので300次元とかのイメージ 
    * なのでそれを受ける一つ目のオプションも300次元の入力を食えるようになっている
    * 300次元は株価方向のラティスをイメージ？
* ひとつめのオプションバリュー自体も300次元であれば株価イメージでいけるがかならずしも同じ次元である必要はない（よね？）
    * その場合の対応するイメージは？？例えば200次元であれば、オプションンバリューのレイヤーは株価方向を粗視化しているとみなすのがいいか？
        *そうだとするとそれってオプションプライシングでも応用可能かも？


# gated RNN

LSTMやGRUなどの総称

soft callのような選択的な期限前償還によりタイムスリップするイメージ
* ただし、満期依存するクーポンでcall notice periodに入るとクーポンのスケジュールもタイムスリップする
* このメカニズムにより非常に遠い満期直前のクーポンの大小が$t=0$におけるオプションバリューに効いてくる


## GRU

(以下、gateの名前が書籍と上記リンクで異なるが書籍のほうを採用する)

* gateによりショートカットしてくるバリュー$n$
    * 普通のRNNやgated RNNとほぼ同じものだが、
    * 一時点前の同レイヤーからのチャンネルがreset gate $r$によって遮断されることがある
* update gate $z$はgated RNNのゲートと同様に機能
*  gate $z$およびupdate gate $r$は普通のRNNのhidden node valueと同じ形式の演算で決まる
    * 書籍の(2.61)式もそのことを物語っている

### option analogy

In [ ]:
* 早期行使が起こるかどうかがupdate gateによってコントロールされている
    * 早期行使

### 以下、古い読解

上記文献の場合、
* candidate node $n$が一番素朴なRNN(活性化関数を$\tanh$としたもの)と自然に対応が付くノード
    * ただし、RNNで素直に一時点前の同じレイヤーの隠れ層からの伝搬があるのに対して、GRUではreset gate $r$がゼロの時は伝搬を遮断する
    * そのreset gateは普通のRNNスタイル（活性化関数はsigmoid）のノード
         * ただし、一時点前のreset gateからの伝搬があるわけではなく、通常のhidden node $h$からの伝搬
* update gate $z$が1になるときは　candidate nodeの値は捨てられて、一時点前の同じレイヤーがそのままスライド採用される
    * これはGated RNNに由来する方法か
    

RNNの設計のうち、オプション的な解釈がむずかしいかもしれないところ
* ひとつめのオプションの現在価値と二つ目のオプションの1ステップ前（すなわち未来）とを混合してから活性化関数にかける
    * ひとつめのオプション価値に依存するクーポンと思えばよいか

# attention

* This is a penというsequenceに二層の隠れ層を用いて、　300x4（4 words $\rightarrow$ 400 x 4 $\rightarrow$ 300 x 4）の出力を得たとする。
* 最後の400および300次元ベクトルがdecoderの隠れ層の初期値となる
    * encoderとdecoderの各隠れ層の次元は同一であるとする
* 翻訳先のdecoderの1単語目の隠れ層出力をまず求める
* 求められた隠れ層出力（300次元ベクトル）とencoderの300次元ベクトル４つそれぞれとの内積を取る
* 内積をウェイトとしたencoderの300次元ベクトルとのweighted sumを求め、もともとのdecoder隠れ層アウトプット300次元とconcatする（600次元になる？）
* 600次元ベクトルを線形変換(600xoutput空間の語彙数)し活性化関数をかける。出力言語の語彙数を10000とすると10000次元のベクトルが出力される
    * 隠れ層がひとつ増える感じ。ただし、普通のRNNの隠れ層ではdecoderの直前ワードの隠れ層をconcatするところ、この最後の「隠れ層」ではattentionがconcatされている

# seq2seq model

* [Seq2Seq in Keras for Petrol Price Prediction using Italian Open Data](https://medium.com/isiway-tech/seq2seq-in-keras-for-petrol-price-prediction-using-italian-open-data-767cb1016af3)はガソリン価格をseq2seqモデルで取り扱う試み


* [High-Dimensional Time Series Forecasting with Convolutional Neural Networks](https://github.com/JEddy92/TimeSeries_Seq2Seq/blob/master/notebooks/TS_Seq2Seq_Conv_Intro.ipynb)

https://jeddy92.github.io/JEddy92.github.io/ts_seq2seq_conv/

# LiNGAM

サンプル次元が列方向になっていることに注意

$$
\begin{align}
x=Bx+e\qquad 
\begin{cases}
B&\text{下三角行列}\\
e &\text{正規分布でない外生変数}
\end{cases}
\end{align}
$$

であるならば$x$の上から順番に因果が流れる

$$
(I-B)x = e
$$

## 識別可能性

```
因果グラフの構造が異なれば、観測変数の分布が異なる
```

場合に識別可能という（？）

## ICA


$$
\begin{align}
x = A s
\qquad
\begin{cases}
x &\text{観測値}\\
s &\text{hidden indemendend components}
\end{cases}
\end{align}
$$

### サンプル相関：
$$
C = \frac{1}{N} x x^{\rm T}
$$

### サンプル相関の固有値分解

$$
\begin{align}
C = U^{\rm T} \Lambda U
\qquad \text{where} 
\begin{cases}
U&\text{直行行列}\\
\Lambda&\text{対角行列}
\end{cases}
\end{align}
$$

### 無相関にする回転

$$
\begin{align}
e =P \Theta V\Lambda^{-1} Ux\qquad \text{where}
\begin{cases}
V&\text{orthgonal matrix}\\
P & \text{permutation}\\
\Theta &\text{diagonal}
\end{cases}
\end{align}
$$


$V$を何らかの方法で特定したとする（？）

上記の$e$がindependent componentと仮定しているので

$$
x = U^{\rm T}\Lambda V^{\rm T} \Theta^{-1}P^{-1}e
$$



上記が
$$
x = (I-B)e
$$
とも書けるはずなので

$$
P\Theta(I-B) = U^{\rm T} \Lambda V^{\rm T} 
$$
右辺を$W_{\rm ICA}$とあらわすことにすると・・

$$
P\Theta(I-B) = W_{\rm ICA}
$$

よってICAで求まった行列にpermutationをかけて、下三角行列にできれば因果の流れが解明できたことになる

## my consideration

* そもそもpermutationくらいで下三角に近くできるとは思いにくい
* ICAはもともと非常にノイズに弱い（はず？）
* Uの自由度を使うのはどうだろうか？

### 無相関性のみ仮定した場合

$$
I-B = U^{\rm T} \Lambda V^{\rm T} \Theta^{-1} P^{-1}
$$

QR分解により、任意の行列は
$$
QR \qquad
\begin{cases}
Q&\text{直行行列}\\
R &\text{上三角行列}
\end{cases}
$$
と書ける

inversibleな行列のQR分解は$R$の対角要素が正であるという条件を置けばユニーク　([wikipedia](https://en.wikipedia.org/wiki/QR_decomposition))

### 因果がわかってなにがうれしいか？ 

To be discussed

# 極値理論

$Z_n$を$n$サンプル
$
X_1, \cdots , X_n
$
の最大値とする

$$
F(x) = P(X_i \le x) \quad\text{for}\quad i = 1,\cdots , n
$$
として、
$$
P(Z_n \le x ) = \Pi_{i=1}^n P(X_i \le x)
={F(x)}^n
$$

$n\to \infty$の極限では　$Z_n$は「$X_i$が取り得る値の最大値が確率１で実現する」ような分布に収束するはず

中心極限定理にある意味似たようなスケーリング則を考えたい
おそらく
$$
\begin{align}
b_n&\to X_i\text{がとり得る値の最大値}\\
a_n &\to 0 \qquad\text{(微視化が進むので拡大倍率を挙げていく必要)}
\end{align}
$$
であるような$a_n,~b_n$で変形した$Z_n$の分布がある確率分布
に収束するとする：
$$
\begin{align}
P\left(
\frac{Z_n - b_n}{a_n} \le y 
\right)\to G(y)
\end{align}
$$


これは
ある$y$を固定したときに
$$
Z_n \le a_n y + b_n
$$
であるような確率が大きい任意の$n$に対して概ね一定であることを意味する

イメージは

* 100サンプルなら最大値が90以下である確率は10%
  $$b_{100} = 100,~a_{100} = 1 ,~  y = -10,~a_{100}y+b_{100}= 90,~ G(-10) = 0.1$$
* 1000サンプルなら最大値が95以下である確率は10%
  $$b_{1000} = 100,~a_{1000} = 0.5 ,~  y = -10,~ a_{1000}y + b_{1000} = 95, ~ G(-10) = 0.1$$
* 10000サンプルなら最大値が99以下である確率は10%
  $$b_{10000} = 100,~a_{10000} = 0.1 ,~  y = -10,~ a_{10000}y + b_{10000} = 99 , G(-10) = 0.1$$

みたいな感じか


書籍の
$$
{F(a_ny+b_n)}^n \to G(y) 
$$

は
$$
\begin{align}
{F(90)}^{100} \approx& G(-10)\\
{F(95)}^{1000}\approx & G(-10)\\
{F(99)}^{10000}\approx & G(-10)
\end{align}
$$
というイメージ。

さらに90,95,99といった閾値以外にも同じ$a_n,b_n$から導かれる99,99.5,99.9といった閾値を最大値が下回る確率についても同じ関係が成り立つことが主張されている

このように極限が$G$で特徴づけられるような確率分布を$G$の吸引領域（domain of attractionと呼ぶ）

# 参考URL

ただしASPECは利用申請がちょっと面倒そう。。

http://www.phontron.com/japanese-translation-data.php?lang=ja



ASPECというcorpusが便利そう

その他のcorpusへのリンク集
https://qiita.com/imaharu/items/9f206a8f470897d5458c